In [1]:
#Libraries
import pandas as pd
import numpy as np
import scanpy as sc
import os
import warnings
import anndata as ad
import scvi
import multiprocessing
import glob

/nemo/stp/babs/working/depottj/.conda/envs/SCVI_conda/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/nemo/stp/babs/working/depottj/.conda/envs/SCVI_conda/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [2]:
#Ignore warnings
warnings.filterwarnings('ignore')

In [3]:
#Scanpy settings
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

scanpy==1.10.1 anndata==0.10.7 umap==0.5.5 numpy==1.26.2 scipy==1.12.0 pandas==2.2.3 scikit-learn==1.4.2 statsmodels==0.14.0 igraph==0.11.5 pynndescent==0.5.11


In [4]:
#SCVI setting
# Check the number of CPU cores
num_cores = multiprocessing.cpu_count()
# To set the number of threads PyTorch will use
scvi.settings.num_threads = (num_cores - 2)
# Set seed
scvi.settings.seed = 1
#To set the progress bar style, choose one of “rich”, “tqdm”
scvi.settings.progress_bar_style = "rich"

Global seed set to 1


In [5]:
#Paths
data_path = "path_to_folder"
adata_output = os.path.join(data_path , "data", "anndata")
adata_seurat_output = os.path.join(data_path , "data", "anndata_seurat")
cesfw_output = os.path.join(data_path , "data", "cesfw" , "preprocessing_larry_chicken")
LARRY_files = os.path.join(data_path , "data", "LARRY_pipeline" , "chicken")
annotation_files = os.path.join(data_path , "data", "cluster_annotation" , "chicken")
meta_files = os.path.join(data_path , "metadata")

In [6]:
#Read in anndata
adata = sc.read_h5ad(os.path.join(adata_output,"preprocessing_larry_chicken.h5ad"))

In [7]:
#Read in CESFW variable features
cesfw_features = np.load(os.path.join(cesfw_output , "selected_cesfw_features_chicken_data.npy") , allow_pickle = True)

In [8]:
#Subset adata with only CESFW genes
adata_cesfw = adata[:,cesfw_features].copy()

In [9]:
#SCVI integration
#Setup anndata
scvi.model.SCVI.setup_anndata(adata_cesfw, batch_key="sample", layer = "counts")

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [10]:
#Setup SCVI model
scvi_model = scvi.model.SCVI(adata_cesfw, n_layers=2, n_latent=30, gene_likelihood="nb")

In [11]:
#Train model
scvi_model.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 234/234: 100%|██████████| 234/234 [06:21<00:00,  1.62s/it, v_num=1, train_loss_step=1.14e+3, train_loss_epoch=1.19e+3]

`Trainer.fit` stopped: `max_epochs=234` reached.


Epoch 234/234: 100%|██████████| 234/234 [06:21<00:00,  1.63s/it, v_num=1, train_loss_step=1.14e+3, train_loss_epoch=1.19e+3]


In [12]:
# Put latent represenation in anndata object
adata.obsm["X_scVI"] = scvi_model.get_latent_representation()

In [13]:
#Graph construction
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=adata.obsm["X_scVI"].shape[1] , use_rep = "X_scVI" , key_added = "scVI_graph",metric = "correlation")

computing neighbors
    finished: added to `.uns['scVI_graph']`
    `.obsp['scVI_graph_distances']`, distances for each pair of neighbors
    `.obsp['scVI_graph_connectivities']`, weighted adjacency matrix (0:00:46)


In [14]:
#Leiden clustering with different resolutions
for i in np.arange(0.2, 8, 0.4):
    #Clustering
    sc.tl.leiden(adata , key_added = "cluster_" + str(int(i*10)), resolution=i, neighbors_key = "scVI_graph")

#Umap
sc.tl.umap(adata , n_components=3 , min_dist = 0.3 , init_pos = "spectral" , neighbors_key = "scVI_graph")

running Leiden clustering
    finished: found 17 clusters and added
    'cluster_2', the cluster labels (adata.obs, categorical) (0:00:18)
running Leiden clustering
    finished: found 30 clusters and added
    'cluster_6', the cluster labels (adata.obs, categorical) (0:00:14)
running Leiden clustering
    finished: found 38 clusters and added
    'cluster_10', the cluster labels (adata.obs, categorical) (0:00:32)
running Leiden clustering
    finished: found 48 clusters and added
    'cluster_14', the cluster labels (adata.obs, categorical) (0:00:33)
running Leiden clustering
    finished: found 54 clusters and added
    'cluster_18', the cluster labels (adata.obs, categorical) (0:00:25)
running Leiden clustering
    finished: found 63 clusters and added
    'cluster_22', the cluster labels (adata.obs, categorical) (0:00:27)
running Leiden clustering
    finished: found 64 clusters and added
    'cluster_26', the cluster labels (adata.obs, categorical) (0:00:30)
running Leiden cluster

IOStream.flush timed out


    finished: found 125 clusters and added
    'cluster_70', the cluster labels (adata.obs, categorical) (0:00:45)
running Leiden clustering
    finished: found 125 clusters and added
    'cluster_74', the cluster labels (adata.obs, categorical) (0:00:33)
running Leiden clustering
    finished: found 130 clusters and added
    'cluster_78', the cluster labels (adata.obs, categorical) (0:00:36)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:23)


In [15]:
#Create meta_data object
meta_data = adata.obs

#Create cell_barcode column
meta_data["cell_barcode"] = [indx.split("-")[0] for indx in meta_data.index]

#Reset index
meta_data = meta_data.reset_index()

In [16]:
#Read in LARRY barcode data
LARRY_output = [pd.read_csv(f) for f in glob.glob(f"{LARRY_files}/*") if os.path.isfile(f)]

In [17]:
#Prepare LARRY barcode data to merge with obs adata
LARRY_data = pd.concat(LARRY_output)
LARRY_data["sample"] = [el.split("_")[0] for el in LARRY_data["Cell"]]
LARRY_data["cell_barcode"] = [el.split(".")[1] for el in LARRY_data["Cell"]]
LARRY_data["clone"] = [clone + "_" + sample for clone , sample in zip(LARRY_data["Clone"],LARRY_data["sample"])]
LARRY_data = LARRY_data.drop(["Clone" , "Cell" , "Count"], axis=1)

In [18]:
#Merge meta and larry data
meta_larry_data = pd.merge(meta_data , LARRY_data, how = "left" , on = ["sample" , "cell_barcode"])

In [19]:
#Read in the cell identities
cluster_identities = pd.read_csv(os.path.join(annotation_files , "preprocessing_larry_chicken_progenitor_neuron.csv"))
cluster_identities["cluster"] = cluster_identities["cluster"].astype("str")

In [20]:
#Merge meta data with the cell identities
meta_identity_data = pd.merge(meta_larry_data , cluster_identities , how = "left" , left_on = "cluster_78", right_on = "cluster")

In [21]:
#Select meta data of interest
meta_identity_data_interest = meta_identity_data[['index','sample','n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo',
       'doublet_score', 'predicted_doublet','rough_type','cell_type','wave','clone']]

In [22]:
#Set meta_larry_data
adata.obs = meta_identity_data_interest.set_index("index")

In [32]:
#Split progenitors
adata_progenitors = adata[adata.obs["rough_type"] == "progenitor"]

#Write out human progenitors
adata_progenitors.write(os.path.join(adata_output,"preprocessing_larry_chicken_progenitors.h5ad"))

#Split neurons
adata_neurons = adata[adata.obs["rough_type"] == "neuron"]

#Write out neuron
adata_neurons.write(os.path.join(adata_output,"preprocessing_larry_chicken_neurons.h5ad"))

### New annotation
From here onwards the clusters of the progenitors + neurons have been annotated.

In [23]:
#Remove old cell_type, wave and rough_type information
meta_identity_data_interest_clean = meta_identity_data_interest.drop(columns = ["rough_type" , "cell_type" , "wave"])

In [24]:
#Read in the cell identities
cell_identities_progenitors = pd.read_csv(os.path.join(annotation_files , "preprocessing_larry_chicken_progenitors_celltypes.csv"))
cell_identities_progenitors["wave"] = "unknown"
cell_identities_neurons = pd.read_csv(os.path.join(annotation_files , "preprocessing_larry_chicken_neurons_celltypes.csv"))
cell_identities = pd.concat([cell_identities_progenitors , cell_identities_neurons])

In [25]:
#Merge meta data with the cell identities
meta_new_cell_identities = pd.merge(meta_identity_data_interest_clean , cell_identities , how = "left" , on = "index")

In [26]:
#Select meta data of interest
meta_identity_data_interest = meta_new_cell_identities[['index','sample','n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo',
       'doublet_score', 'predicted_doublet','cell_type','wave','clone']]

In [27]:
#Set meta_larry_data
adata.obs = meta_identity_data_interest.set_index("index")

In [28]:
#Write meta data out
adata.obs.reset_index().to_csv(os.path.join(meta_files,"larry_chicken.csv"), index = False)

In [29]:
#Write out human clusters
adata.write(os.path.join(adata_output,"preprocessing_larry_chicken_clusters.h5ad"))

In [30]:
#Write out count matrix as csv file
import scipy.io
scipy.io.mmwrite(os.path.join(adata_seurat_output,"preprocessing_larry_chicken_matrix.mtx"), adata.layers["counts"])

#Write out gene names
gene_names_df = pd.DataFrame(adata.var_names, columns=["Gene"])
gene_names_df.to_csv(os.path.join(adata_seurat_output,"preprocessing_larry_chicken_genes.csv"), index=False)

#Write out cell names
cell_names_df = pd.DataFrame(list(adata.obs_names), columns=["Cells"])
cell_names_df.to_csv(os.path.join(adata_seurat_output,"preprocessing_larry_chicken_cells.csv"), index=False)

#Write out meta data
adata.obs.to_csv(os.path.join(adata_seurat_output,"preprocessing_larry_chicken_meta.csv"))

#Write out umap
umap = pd.DataFrame(adata.obsm["X_umap"])
umap.to_csv(os.path.join(adata_seurat_output,"preprocessing_larry_chicken_umap.csv"), index=False, header=False)